# Advent of Code - Day 8

## Reading data

In [17]:
using BenchmarkTools

In [19]:
function bla(a,b,c)
    return a + b + c
end

bla (generic function with 1 method)

In [20]:
bla(1,2,3)

6

In [21]:
input = [1,2,3]
bla(input...)

6

In [22]:
mat = parse.(Int,hcat(split.(readlines("input"),"")...))'

99×99 adjoint(::Matrix{Int64}) with eltype Int64:
 0  0  2  1  0  0  0  3  0  2  2  2  3  …  1  0  1  2  2  3  1  2  0  1  1  0
 1  0  2  0  3  0  3  2  2  3  4  1  1     1  1  0  3  0  2  3  2  0  0  0  3
 0  0  3  0  1  2  2  0  3  4  0  4  0     2  4  3  3  1  4  1  0  2  0  3  3
 3  3  0  3  1  3  4  2  3  0  0  2  0     0  1  3  4  3  0  1  4  1  2  0  3
 3  3  1  2  0  3  1  2  1  1  3  1  2     3  1  0  1  3  4  3  2  4  3  0  0
 2  3  2  2  0  1  2  3  1  0  2  1  0  …  2  4  0  4  1  1  2  1  3  4  3  1
 0  3  3  0  3  3  3  3  3  3  0  1  0     2  3  2  2  0  1  2  3  0  4  4  1
 1  3  2  2  0  1  0  3  2  2  3  4  3     4  5  3  0  4  1  1  1  1  2  3  4
 3  1  4  3  2  2  3  4  1  3  2  2  1     5  1  4  0  1  4  2  3  4  3  4  4
 1  0  1  3  2  3  0  4  3  0  2  3  4     2  5  3  1  0  2  2  1  1  3  1  0
 3  2  0  4  0  4  2  4  2  5  1  4  1  …  3  2  2  0  3  2  4  2  1  3  0  3
 2  2  1  2  0  4  1  2  3  2  0  5  1     5  5  4  3  3  3  1  0  2  0  3  4
 3  4  0  3  1

## Part 1

### Helper Functions

In [23]:
function is_border(mat, i, j)
    border_row = i in [1,size(mat,1)]
    border_col = j in [1,size(mat,2)]
    return border_row || border_col
end;

In [24]:
function get_views(mat, i, j)
    left, right = mat[i,1:j-1], mat[i,j+1:end]
    top, bottom = mat[1:i-1,j], mat[i+1:end,j]
    return left, right, top, bottom
end;

In [25]:
function is_visible(mat, i, j)
    if is_border(mat,i,j)
        return true
    end
    center = mat[i,j]
    minimax = minimum(maximum.(get_views(mat,i,j)))
    return center > minimax
end;

In [26]:
function all_positions(mat)
    return Iterators.product(1:size(mat,1), 1:size(mat,2))
end;

### Solution

In [27]:
@benchmark sum([is_visible(mat,i,j) for (i,j) in all_positions(mat)])

BenchmarkTools.Trial: 340 samples with 1 evaluation.
 Range (min … max):   6.556 ms … 30.695 ms  ┊ GC (min … max):  0.00% … 33.21%
 Time  (median):     14.020 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   14.666 ms ±  4.118 ms  ┊ GC (mean ± σ):  16.61% ± 17.04%

               ▄▃█▄▂▃▁▂ ▁  ▃▄▄▂▁                               
  ▄▃▃▃▄▄▅▇▆███▇████████▆█▆██████▆██▅▆▆▃▆▅▃▃▆▁▄▃▃▃▃▃▄▁▃▃▁▁▃▁▁▃ ▄
  6.56 ms         Histogram: frequency by time        27.3 ms <

 Memory estimate: 18.19 MiB, allocs estimate: 57245.

## Part 2

### Helper Functions

In [28]:
function get_directed_views(mat,i,j)
    left, right, top, bottom = get_views(mat,i,j)
    return reverse(left), right, reverse(top), bottom
end;

In [29]:
function get_viewing_dist(dview, height)
    blocker = findfirst(dview .>= height)
    if blocker |> isnothing
        return length(dview)
    end
    return blocker
end;

In [30]:
function scenic_score(mat, i, j)
    dviews = get_directed_views(mat, i, j)
    dists = get_viewing_dist.(dviews, mat[i,j])
    return prod(dists)
end;

### Solution

In [32]:
@benchmark begin
    mat = parse.(Int,hcat(split.(readlines("input"),"")...))'
    sum([is_visible(mat,i,j) for (i,j) in all_positions(mat)])
    maximum([scenic_score(mat,i,j) for (i,j) in all_positions(mat)])
end

BenchmarkTools.Trial: 103 samples with 1 evaluation.
 Range (min … max):  24.947 ms … 99.364 ms  ┊ GC (min … max): 12.00% … 15.43%
 Time  (median):     45.314 ms              ┊ GC (median):    17.96%
 Time  (mean ± σ):   48.679 ms ± 14.545 ms  ┊ GC (mean ± σ):  18.88% ±  7.95%

          ▂     ▂█     ▂                                       
  ▅▃▃▁▅▆▃▆█▅▃███████▆▅██▃█▃▁▃▃▁█▅▃▅▃▃▁▃▆▁▃▃▁▁▃▁▁▁▃▁▃▁▁▁▁▁▁▁▁▃ ▃
  24.9 ms         Histogram: frequency by time        97.1 ms <

 Memory estimate: 50.20 MiB, allocs estimate: 214885.